In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2023-12-19 11:41:09.733941: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 11:41:09.733969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 11:41:09.734718: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from tqdm import tqdm

weights=models.ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1
model = models.vit_b_16(weights=weights)
preprocess = weights.transforms()
model.heads[-1] = nn.Linear(model.heads[-1].in_features, 29)
model.load_state_dict(torch.load('./models/vitb16_SWAGE2E_model.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Load the YOLO model
yolo_model = YOLO('./YOLOv8_training/best.pt').to(device)

# Class mapping
class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './datasets/test/images'
# directory = './TEST'

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
preds=[]
# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

In [5]:
df = pd.read_csv('./YOLOv8_training/new_labels.csv')
# df = pd.read_csv('./set.csv')
model.eval()
# Define the target size for ResNet50
target_size = (224, 224)
# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory,filename))
    for x in results:
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)
            input_image = preprocess(cropped_im).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(input_image)
            logits = torch.nn.functional.softmax(output[0], dim=0)
            # model predicts one of the 1000 ImageNet classes
            predicted_class_index = torch.argmax(logits).item()
            predicted_class_label= list(class_mapping.keys())[predicted_class_index]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([predicted_class_index]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([logits[predicted_class_index].item()]).to(device)*r.boxes.conf
#             score_tensor=r.boxes.conf
            minind=0
            q=0
            err=1e9
            for(index, row) in filtered_df.iterrows():
                # Access fields using the column names
                temerr=(bbox[0]-row['xmin'])**2+(bbox[1]-row['ymin'])**2+(bbox[2]-row['xmax'])**2+(bbox[3]-row['ymax'])**2
                if(temerr<err):
                    err=temerr
                    minind=q
                q+=1
            print(len(filtered_df), " ", minind)
            if(len(filtered_df)>0):
                data.append({'boxes': torch.tensor([[(filtered_df.iloc[minind]['xmin'].copy()), (filtered_df.iloc[minind]['ymin'].copy()), (filtered_df.iloc[minind]['xmax'].copy()), (filtered_df.iloc[minind]['ymax'].copy())]]).to(device), 'labels': torch.tensor([(class_mapping[filtered_df.iloc[minind]['class']])]).to(device)})
                # filtered_df.drop(filtered_df.index[minind], inplace=True)
                preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
    filtered_df=[]
    
    


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2458.jpg: 480x640 12 leafs, 79.8ms
Speed: 2.5ms preprocess, 79.8ms inference, 310.5ms postprocess per image at shape (1, 3, 480, 640)
10   4
10   7
10   6
10   9
10   0
10   8
10   5
10   0
10   2
10   7
10   3
10   8

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img638.jpg: 576x640 1 leaf, 88.8ms
Speed: 1.5ms preprocess, 88.8ms inference, 0.9ms postprocess per image at shape (1, 3, 576, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img454.jpg: 544x640 1 leaf, 87.1ms
Speed: 1.2ms preprocess, 87.1ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1086.jpg: 416x640 2 leafs, 78.1ms
Speed: 2.3ms preprocess, 78.1ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)
2   1
2   0

imag

5   2
5   2
5   3
5   4
5   4
5   0
5   2
5   0
5   2
5   0
5   0
5   0
5   0
5   0
5   1
5   2
5   1
5   2
5   0
5   2
5   0
5   0
5   2
5   2
5   1
5   4
5   1
5   0
5   1
5   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img659.jpg: 512x640 3 leafs, 7.4ms
Speed: 1.5ms preprocess, 7.4ms inference, 0.7ms postprocess per image at shape (1, 3, 512, 640)
2   1
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1171.jpg: 480x640 19 leafs, 11.5ms
Speed: 2.9ms preprocess, 11.5ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
9   0
9   0
9   7
9   6
9   2
9   6
9   7
9   4
9   3
9   7
9   1
9   1
9   3
9   5
9   6
9   3
9   0
9   0
9   6

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1538.jpg: 480x640 8 leafs, 7.6ms
Speed: 1.3ms preprocess, 7.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
9   1
9   6
9   5
9   2
9 

1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1534.jpg: 640x640 2 leafs, 12.9ms
Speed: 2.2ms preprocess, 12.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img125.jpg: 480x640 1 leaf, 9.2ms
Speed: 1.3ms preprocess, 9.2ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1584.jpg: 480x640 1 leaf, 12.6ms
Speed: 2.4ms preprocess, 12.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img287.jpg: 608x640 43 leafs, 8.2ms
Speed: 1.6ms preprocess, 8.2ms inference, 0.6ms postprocess per image at shape (1, 3, 608, 640)
17   7
17   12
17   12
17   5
17   14
17   14
17   13
17   12
17   10
17   5
17   13
17  

Speed: 1.9ms preprocess, 12.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
1   0
1   0
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1136.jpg: 512x640 1 leaf, 7.5ms
Speed: 1.3ms preprocess, 7.5ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1789.jpg: 640x640 10 leafs, 10.1ms
Speed: 1.5ms preprocess, 10.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
10   8
10   6
10   9
10   2
10   7
10   0
10   3
10   6
10   7
10   3

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2145.jpg: 640x640 5 leafs, 12.6ms
Speed: 1.5ms preprocess, 12.6ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)
5   3
5   4
5   2
5   0
5   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img388.jpg: 480

7   2
7   0
7   5
7   6
7   3
7   3
7   3
7   5
7   1
7   1
7   4
7   0
7   6
7   3

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1100.jpg: 544x640 5 leafs, 8.3ms
Speed: 1.5ms preprocess, 8.3ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 640)
5   0
5   2
5   1
5   3
5   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img135.jpg: 416x640 1 leaf, 6.8ms
Speed: 1.7ms preprocess, 6.8ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img326.jpg: 640x640 2 leafs, 15.1ms
Speed: 2.0ms preprocess, 15.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1942.jpg: 640x544 1 leaf, 9.9ms
Speed: 1.3ms preprocess, 9.9ms inference, 0.7ms postprocess per image at shape (1, 3, 

Speed: 1.6ms preprocess, 15.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1568.jpg: 384x640 2 leafs, 7.0ms
Speed: 1.7ms preprocess, 7.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2034.jpg: 448x640 1 leaf, 7.1ms
Speed: 2.4ms preprocess, 7.1ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1233.jpg: 448x640 4 leafs, 6.7ms
Speed: 3.2ms preprocess, 6.7ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
1   0
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img915.jpg: 480x640 27 leafs, 8.6ms
Speed: 1.2ms preprocess, 8.6ms inference, 0.7ms postprocess per image at

Speed: 1.7ms preprocess, 8.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
33   0
33   12
33   24
33   23
33   15
33   29
33   9
33   18
33   9
33   28
33   15
33   10
33   29
33   4
33   19
33   1
33   26
33   6
33   20
33   8
33   18
33   25
33   14
33   5
33   26
33   13
33   16
33   27
33   3
33   12
33   19
33   31
33   8
33   15

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img346.jpg: 544x640 5 leafs, 9.1ms
Speed: 1.5ms preprocess, 9.1ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 640)
5   2
5   3
5   1
5   4
5   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img924.jpg: 640x640 9 leafs, 10.8ms
Speed: 1.7ms preprocess, 10.8ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)
7   5
7   2
7   1
7   3
7   5
7   4
7   5
7   0
7   5

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img208.jpg: 640x480 1 leaf, 10

Speed: 2.1ms preprocess, 10.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
3   0
3   1
3   2
3   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img0.jpg: 480x640 3 leafs, 8.1ms
Speed: 1.4ms preprocess, 8.1ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img983.jpg: 640x480 4 leafs, 11.3ms
Speed: 1.9ms preprocess, 11.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
5   0
5   1
5   3
5   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1229.jpg: 480x640 1 leaf, 8.1ms
Speed: 1.1ms preprocess, 8.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1443.jpg: 640x384 1 leaf, 5.9ms
Speed: 1.0ms preprocess, 5.9ms inference, 0.6ms pos

In [6]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [7]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(preds)):
    print([preds[i]], [data[i]])
    metrics.update([preds[i]], [data[i]])
print(metrics.compute())

[{'boxes': tensor([[305.9838, 293.8942, 437.3342, 479.8495]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.8399], device='cuda:0')}] [{'boxes': tensor([[312, 288, 445, 475]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[ 14.0796, 155.6635, 212.2745, 286.5225]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.5446], device='cuda:0')}] [{'boxes': tensor([[ 25, 160, 204, 279]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[209.6182, 224.8037, 342.9072, 447.2449]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.4186], device='cuda:0')}] [{'boxes': tensor([[215, 240, 323, 430]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[238.8710,  19.7182, 340.7936, 225.7074]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.4839], device='cuda:0')}] [{'boxes': tensor([[249,  1

[{'boxes': tensor([[5.2195e+02, 3.2852e-01, 7.4200e+02, 3.0277e+02]], device='cuda:0'), 'labels': tensor([12], device='cuda:0'), 'scores': tensor([0.4994], device='cuda:0')}] [{'boxes': tensor([[523,   6, 742, 286]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}]
[{'boxes': tensor([[ 24.2101,   4.2984, 535.9792, 469.1835]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.8397], device='cuda:0')}] [{'boxes': tensor([[175,   3, 543, 471]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,   7.1465,  69.0984, 463.9371]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.1730], device='cuda:0')}] [{'boxes': tensor([[ 45,   3, 188, 214]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}]
[{'boxes': tensor([[ 366.1549,  242.7937, 1891.4818, 1230.7393]], device='cuda:0'), 'labels': tensor([12], device='cuda:0'), 'scores': tensor([0.8887], device='cuda:0')}] [{'boxes': ten

[{'boxes': tensor([[1.5980e+02, 2.4563e-02, 3.1970e+02, 5.0296e+01]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.6672], device='cuda:0')}] [{'boxes': tensor([[ 34,  17, 215, 196]], device='cuda:0'), 'labels': tensor([28], device='cuda:0')}]
[{'boxes': tensor([[0.0000e+00, 1.7441e-02, 6.4617e+01, 1.3956e+02]], device='cuda:0'), 'labels': tensor([4], device='cuda:0'), 'scores': tensor([0.2056], device='cuda:0')}] [{'boxes': tensor([[ 34,  17, 215, 196]], device='cuda:0'), 'labels': tensor([28], device='cuda:0')}]
[{'boxes': tensor([[107.3035, 187.0525, 270.0575, 240.0000]], device='cuda:0'), 'labels': tensor([14], device='cuda:0'), 'scores': tensor([0.2550], device='cuda:0')}] [{'boxes': tensor([[ 34,  17, 215, 196]], device='cuda:0'), 'labels': tensor([28], device='cuda:0')}]
[{'boxes': tensor([[ 991.9684,    0.0000, 1499.9802,  717.1795]], device='cuda:0'), 'labels': tensor([16], device='cuda:0'), 'scores': tensor([0.8759], device='cuda:0')}] [{'boxe

[{'boxes': tensor([[480.3751, 138.7005, 771.5746, 339.3118]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.8809], device='cuda:0')}] [{'boxes': tensor([[481, 140, 759, 350]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[123.0376,  41.2537, 337.6043, 223.8407]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.8625], device='cuda:0')}] [{'boxes': tensor([[123,  42, 352, 220]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[ 25.4237, 194.4619, 325.4926, 362.8388]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.8495], device='cuda:0')}] [{'boxes': tensor([[ 23, 200, 327, 366]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[247.9723, 261.9701, 412.7121, 481.7600]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.8437], device='cuda:0')}] [{'boxes': tensor([[249, 2

[{'boxes': tensor([[581.5161, 192.8902, 669.1857, 326.1030]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.5450], device='cuda:0')}] [{'boxes': tensor([[591, 200, 665, 290]], device='cuda:0'), 'labels': tensor([3], device='cuda:0')}]
[{'boxes': tensor([[314.5436, 293.6790, 427.8157, 362.5063]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.5670], device='cuda:0')}] [{'boxes': tensor([[326, 291, 431, 362]], device='cuda:0'), 'labels': tensor([3], device='cuda:0')}]
[{'boxes': tensor([[384.8268, 162.1970, 466.2854, 267.0978]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.2814], device='cuda:0')}] [{'boxes': tensor([[391, 165, 450, 259]], device='cuda:0'), 'labels': tensor([3], device='cuda:0')}]
[{'boxes': tensor([[514.3174, 159.1511, 602.4615, 258.7216]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.4128], device='cuda:0')}] [{'boxes': tensor([[442, 161, 5

[{'boxes': tensor([[1617.4904, 2656.1836, 1793.8247, 2971.2876]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.3256], device='cuda:0')}] [{'boxes': tensor([[1171, 2457, 1667, 3000]], device='cuda:0'), 'labels': tensor([20], device='cuda:0')}]
[{'boxes': tensor([[2069.3794, 1232.8009, 2687.7708, 1537.2103]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.3596], device='cuda:0')}] [{'boxes': tensor([[2071, 1242, 2671, 1595]], device='cuda:0'), 'labels': tensor([20], device='cuda:0')}]
[{'boxes': tensor([[1134.1202,  489.0770, 1409.0825,  692.8104]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.3588], device='cuda:0')}] [{'boxes': tensor([[1123,  632, 1604,  885]], device='cuda:0'), 'labels': tensor([20], device='cuda:0')}]
[{'boxes': tensor([[ 113.0638, 1491.1642,  564.0999, 1911.8925]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.3481], device='cuda:0')}

{'map': tensor(0.2517), 'map_50': tensor(0.3546), 'map_75': tensor(0.2810), 'map_small': tensor(0.2618), 'map_medium': tensor(0.1530), 'map_large': tensor(0.2760), 'mar_1': tensor(0.3191), 'mar_10': tensor(0.3191), 'mar_100': tensor(0.3191), 'mar_small': tensor(0.4189), 'mar_medium': tensor(0.2095), 'mar_large': tensor(0.3418), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28], dtype=torch.int32)}
